## 6_Sprint_6T01
### Tasca mètodes de mostreig
### Eloi Canals v1

In [43]:
# imports
import pandas as pd
import numpy as np

In [44]:
# proves amb MLB
mlb = pd.read_csv('mlb.txt', sep = "	")
mlb.head()

,player,team,position,salary
0,Brandon Webb,Arizona Diamondbacks,Pitcher,8500.0
1,Danny Haren,Arizona Diamondbacks,Pitcher,8250.0
2,Chris Snyder,Arizona Diamondbacks,Catcher,5250.0
3,Edwin Jackson,Arizona Diamondbacks,Pitcher,4600.0
4,Adam LaRoche,Arizona Diamondbacks,First Baseman,4500.0


In [45]:
mlb.describe()

,salary
count,828.000000
mean,3281.828339
std,4465.448980
min,400.000000
25%,418.315000
50%,1093.750000
75%,4250.000000
max,33000.000000


In [46]:
# Farem un sampling del 20%
total = int(len(mlb))
mostra_perc = int(20 * total / 100)
mostra_perc 

165

#### Exercici 1. Mostra aleatoria simple

In [47]:
mostra_simple_df = mlb.sample(n = mostra_perc).sort_index(ascending = True)
mostra_simple_df

,player,team,position,salary
1,Danny Haren,Arizona Diamondbacks,Pitcher,8250.0
8,Conor Jackson,Arizona Diamondbacks,First Baseman,3100.0
35,Matt Diaz,Atlanta Braves,Outfielder,2550.0
41,Jair Jurrjens,Atlanta Braves,Pitcher,480.0
42,Eric O'Flaherty,Atlanta Braves,Pitcher,440.0
...,...,...,...,...
795,Randy Ruiz,Toronto Blue Jays,First Baseman,404.3
802,Josh Willingham,Washington Nationals,Outfielder,4600.0
805,Chien-Ming Wang,Washington Nationals,Pitcher,2000.0
814,Chris Coste,Washington Nationals,Catcher,650.0


#### Exercici 1b. Mostra sistemàtica

In [48]:
# Definim aleatoriament l'inici i el interval. 


def inici_interval(total, num_mostres):
    # Funció recursiva per obtenir inici i interval que estigui inclòs en el total de casos del dataset
    inici = np.random.randint(1, total)
    interval = np.random.randint(2, total/20)
    mytry = inici + (interval * num_mostres)
    if  mytry > total:
        print(f"DISCARTED. Començant desde {inici}, fent {num_mostres} samplings en intervals de {interval}, se superaria el total de casos ({total}) del dataset.")
        return inici_interval(total,num_mostres)
    else:
        return inici,interval

inici, interval = inici_interval(total, mostra_perc)
print(f"Els numeros aleatoris correctes son: començar desde {inici}, en intervals de {interval}.")

DISCARTED. Començant desde 578, fent 165 samplings en intervals de 19, se superaria el total de casos (828) del dataset.
DISCARTED. Començant desde 636, fent 165 samplings en intervals de 25, se superaria el total de casos (828) del dataset.
DISCARTED. Començant desde 284, fent 165 samplings en intervals de 27, se superaria el total de casos (828) del dataset.
DISCARTED. Començant desde 684, fent 165 samplings en intervals de 24, se superaria el total de casos (828) del dataset.
DISCARTED. Començant desde 172, fent 165 samplings en intervals de 31, se superaria el total de casos (828) del dataset.
DISCARTED. Començant desde 336, fent 165 samplings en intervals de 3, se superaria el total de casos (828) del dataset.
DISCARTED. Començant desde 494, fent 165 samplings en intervals de 17, se superaria el total de casos (828) del dataset.
DISCARTED. Començant desde 731, fent 165 samplings en intervals de 6, se superaria el total de casos (828) del dataset.
DISCARTED. Començant desde 506, fe

In [49]:
# mostra sistematica
mostra_sistematica_df = mlb[inici : total : interval]
mostra_sistematica_df

,player,team,position,salary
112,Ryan Dempster,Chicago Cubs,Pitcher,13500.0
114,Ted Lilly,Chicago Cubs,Pitcher,13000.0
116,Xavier Nady,Chicago Cubs,Outfielder,3300.0
118,Jeff Samardzija,Chicago Cubs,Pitcher,3000.0
120,Ryan Theriot,Chicago Cubs,Shortstop,2600.0
...,...,...,...,...
818,Alberto Gonzalez,Washington Nationals,Shortstop,415.5
820,Mike Morse,Washington Nationals,First Baseman,410.0
822,Tyler Clippard,Washington Nationals,Pitcher,401.0
824,Ian Desmond,Washington Nationals,Shortstop,400.0


#### Exercici 2. Generació una mostra estratificada i una mostra utilitzant SMOTE (Synthetic Minority Oversampling Technique).

#### En aquest cas voldrem saber el salary dels jugadors estratificat
#### per tipus de posicio en el camp, per veure si hi ha posicions més gratificades,
#### pero per tots els equips representats.

Primer hem d'averiguar el percentatge de jugadors que hi ha a cada position per poder agafar la mostra de forma correcta



In [50]:
player_types = mlb.position.value_counts()
player_types

Pitcher              410
Outfielder           148
Catcher               67
Shortstop             56
Second Baseman        48
Third Baseman         46
First Baseman         42
Designated Hitter      7
Infielder              4
Name: position, dtype: int64

In [51]:
percent = player_types * 100 / total
percent

Pitcher              49.516908
Outfielder           17.874396
Catcher               8.091787
Shortstop             6.763285
Second Baseman        5.797101
Third Baseman         5.555556
First Baseman         5.072464
Designated Hitter     0.845411
Infielder             0.483092
Name: position, dtype: float64

In [52]:
percent.sum()

100.00000000000001

El percentatge de samples per categoria respecte el total de jugadors es:
(El sampling total sera del 20% del total del dataset)

In [53]:
percent_samples_player_category = round(((total  * 20) / 100) * percent / 100)
percent_samples_player_category

Pitcher              82.0
Outfielder           30.0
Catcher              13.0
Shortstop            11.0
Second Baseman       10.0
Third Baseman         9.0
First Baseman         8.0
Designated Hitter     1.0
Infielder             1.0
Name: position, dtype: float64

In [54]:
percent_samples_player_category.sum()

165.0

#### Efectuem el sampling

In [56]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest = train_test_split(mlb, test_size = 0.20, stratify = mlb[['position']])
Xtest.count()

player      166
team        166
position    166
salary      166
dtype: int64

In [57]:
Xtest

,player,team,position,salary
43,Martin Prado,Atlanta Braves,Second Baseman,440.0
330,Kyle Farnsworth,Kansas City Royals,Pitcher,4500.0
437,Josh Butler,Milwaukee Brewers,Pitcher,400.0
617,Clayton Richard,San Diego Padres,Pitcher,423.7
737,Reid Brignac,Tampa Bay Rays,Shortstop,403.7
...,...,...,...,...
1,Danny Haren,Arizona Diamondbacks,Pitcher,8250.0
11,Bobby Howry,Arizona Diamondbacks,Pitcher,2000.0
721,Kelly Shoppach,Tampa Bay Rays,Catcher,2250.0
560,Shane Victorino,Philadelphia Phillies,Outfielder,5000.0


#### Exercici 2B. Mostra utilitzant SMOTE (Synthetic Minority Oversampling Technique).

In [59]:
import imblearn
from imblearn.over_sampling import SMOTE

In [60]:
oversample = SMOTE(sampling_strategy='not majority', k_neighbors=3, random_state=6858)

In [61]:
player_over, position_over = oversample.fit_resample(mlb[['salary']], mlb[['position']])

In [68]:
position_over.value_counts()

position         
Catcher              410
Designated Hitter    410
First Baseman        410
Infielder            410
Outfielder           410
Pitcher              410
Second Baseman       410
Shortstop            410
Third Baseman        410
dtype: int64

In [69]:
player_over['position'] = position_over

In [70]:
mitja_real = round(mlb.salary.mean(),3)
stratified_SMOTE=(player_over.groupby('position').apply(lambda x: x.sample(n=100, random_state=6858)).reset_index(level=0, drop=True))

In [71]:
abs(stratified_SMOTE.salary.mean()-mitja_real) / stratified_SMOTE.salary.mean()*100

0.8386615543679513

In [72]:
stratified_SMOTE.position.value_counts(normalize=True)

Outfielder           0.111111
Pitcher              0.111111
Designated Hitter    0.111111
Catcher              0.111111
Infielder            0.111111
Third Baseman        0.111111
Shortstop            0.111111
Second Baseman       0.111111
First Baseman        0.111111
Name: position, dtype: float64

In [67]:
stratified_SMOTE

,salary,position
387,5050.000000,Catcher
1131,2060.721728,Catcher
885,400.128833,Catcher
905,455.396385,Catcher
1084,1397.322518,Catcher
...,...,...
3459,5080.798208,Third Baseman
3657,10173.018667,Third Baseman
3432,6314.134796,Third Baseman
3517,895.809512,Third Baseman


#### Exercici 3. Mostra utilitzant el mètodo Reservoir Sampling

Sobre el paper, el Reservoir Sampling es adient per fer samplings sobre un caudal infinit de mostres. En aquest cas el caudal es finit i el numero de samplings a realitzar es defineix d'una manera arbitraria. Com 'no sabem' quants jugadors vidran per el caudal, es defineix com a numero de samplings a fer, el total de jugadors que componen un equip estandard de baseball de la lliga MLB : 28 com a maxim.
https://make-shots.com/players-baseball-team/

In [34]:
import random

stream = mlb.index
# Doing Reservoir Sampling from the stream
k=28
reservoir = []
for i, element in enumerate(stream):
    if i+1<= k:
        reservoir.append(element)
    else:
        probability = k/(i+1)
        if random.random() < probability:
            # Select item in stream and remove one of the k items already selected
             reservoir[random.choice(range(0,k))] = element


In [35]:
reservoir

[630,
 662,
 30,
 242,
 585,
 568,
 735,
 254,
 382,
 528,
 346,
 207,
 394,
 467,
 449,
 339,
 323,
 728,
 587,
 235,
 299,
 424,
 777,
 45,
 530,
 257,
 820,
 821]

In [39]:
reservoir_mlb_sampling = mlb[mlb.index.isin(reservoir)]
reservoir_mlb_sampling

,player,team,position,salary
30,Billy Wagner,Atlanta Braves,Pitcher,6750.000
45,Tommy Hanson,Atlanta Braves,Pitcher,435.000
207,Luis Valbuena,Cleveland Indians,Infielder,414.700
235,Randy Flores,Colorado Rockies,Pitcher,650.000
242,Greg Reynolds,Colorado Rockies,Pitcher,403.000
254,Bobby Seay,Detroit Tigers,Pitcher,2475.000
257,Max Scherzer,Detroit Tigers,Pitcher,1500.000
299,Roy Oswalt,Houston Astros,Pitcher,15000.000
323,Yorman Bazardo,Houston Astros,Pitcher,400.000
339,Willie Bloomquist,Kansas City Royals,Second Baseman,1700.000
